##### 0. Notebook setup and imports <br>
Here we make the preparation necessary for the notebook to run

In [ ]:
# 0.1 Notebook setup
import sys
import os
print(sys.path)

NOTEBOOK_DIR = os.getcwd()
print(NOTEBOOK_DIR)
ROOT_DIR = os.path.dirname(NOTEBOOK_DIR)
print(ROOT_DIR)
SRC_DIR = os.path.join(ROOT_DIR, 'src')
print(SRC_DIR)

# Adds project's roof to sys.path
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\python311.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0', '', 'C:\\Users\\Claudia\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages', 'C:\\Users\\Claudia\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\win32', 'C:\\Users\\Claudia\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\Claudia\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Pyth

In [ ]:
# 0.2 Imports
import requests                   # Allows me to make requests from web pages
import sqlite3                    # Chosen SQL database
import pandas as pd
from datetime import datetime, timedelta    # Helps us convert the UNIX time into legible format and vice-versa
import time                       #Allows us to have the now() method
from indicators import calculate_sma, calculate_returns_std, calculate_atr, calculate_bollinger_bands

##### 1. Preparing table and data for the API

a. Checks if the table exists (if not, we create one)  
b. Verifies the last existing date in the table.  
c. Converts the date from string to a datetime format

In [ ]:
# 1.1 Creating connection

connection = sqlite3.connect('../data/crypto.db') #connects Python to SQLite
cursor = connection.cursor()

In [ ]:
# 1.2 Guarantees the table exists

cursor.execute ("""
    CREATE TABLE IF NOT EXISTS bitcoin_prices (
                date DATETIME PRIMARY KEY NOT NULL,
                open REAL NOT NULL,
                high REAL NOT NULL,
                low REAL NOT NULL,
                close REAL NOT NULL,
                volume REAL NOT NULL
                )
""")

connection.commit()

In [ ]:
# 1.3 Checks last date inserted into the table

cursor.execute ("""
    SELECT MAX(date)
    FROM bitcoin_prices
""")

last_date_str = cursor.fetchone()[0]
print(last_date_str)

2025-04-30 17:00:00


In [ ]:
# 1.4 Sets a readable last_date datetime variable to the API
if last_date_str is None:
    last_date = datetime.strptime("2024-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")
else:
    last_date = datetime.strptime(last_date_str, "%Y-%m-%d %H:%M:%S")

    print(((last_date)))


2025-04-30 17:00:00


##### 2. Requesting data from the API  

a. Converts the 'last date' datetime format to a unix format, so the API undestands
b. Requests the data from Coin Desk's API and then loops over it, feeding it into the SQL table bitcoin_prices, inside the crypto.db database


In [ ]:
# 2.1 Calling the API for new data
limit = 1000
inserted_candles = 0
last_date = datetime.strptime(last_date_str, "%Y-%m-%d %H:%M:%S")

while True:

    # Defines timestamp
    from_last_date = last_date
    to_dt = from_last_date + timedelta(hours=limit-1)
    to_ts = int(to_dt.timestamp())

    # CryptoCompare URL
    CC_URL = "https://data-api.coindesk.com/index/cc/v1/historical/hours"

    # Parameters required to request the data
    params={
        "market":"cadli",                                       # REQUIRED FIELD! This one is a default market index, but you can pick many others
        "instrument":"BTC-USD",                                 # REQUIRED FIELD! It's the trading pair
        "limit": limit,                                          # Number of datapoints returned
        "aggregate": 1,                                         # Defines how you'll group data. In this case, I've choosen the 1 hour timespan
        "fill":"true",                                          # Boolean value. True has it return datapoints even in periods with no trading activity
        "apply_mapping":"true",                                 # Determines if provided instrument values are converted according to internal mappings. (if the pair was "USD-BTC", the true value would convert the results accordingly
        "response_format":"JSON",                               # It may be defined as JSON or CSV
        "to_ts" : to_ts,                                          # Defines a start date
        }

    # Getting response and recording it in a variable
    response = requests.get(CC_URL, params=params, headers={"Content-type":"application/json; charset=UTF-8"})
    bitcoin_data = response.json()
    
    if "Data" in bitcoin_data and isinstance(bitcoin_data["Data"], list):
        new_rows = 0

        for item in bitcoin_data["Data"]:
            to_now = ((datetime.now()))
            now = int(to_now.timestamp())
            last_date_ts =int(last_date.timestamp())
            ts = item["TIMESTAMP"]
            dt = datetime.fromtimestamp(ts)
            
            if ts <= last_date_ts or ts >= now:
                continue
        
            open_p = item["OPEN"]
            high_p = item["HIGH"]
            low_p = item["LOW"]
            close_p = item["CLOSE"]
            volume = item["VOLUME"]

            cursor.execute("""
                INSERT OR IGNORE INTO bitcoin_prices
                (date, open, high, low, close, volume)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (dt, open_p, high_p, low_p, close_p, volume))

            new_rows += 1
            last_date = dt

        connection.commit()
        inserted_candles += new_rows
        print(f"✅ Table successfuly updated with {new_rows} new entries.")
        
        if new_rows == 0:
            print("❌ No new candle found. Shutting down.")
            break
        else:
            time.sleep(1.2)
    else:
        print("⚠️ No new data returned from the API.")
        break


print(f"Total candles inserted: {inserted_candles}")

✅ Table successfuly updated with 1 new entries.
✅ Table successfuly updated with 0 new entries.
❌ No new candle found. Shutting down.
Total candles inserted: 1


In [ ]:
# 2.2 Checking the SQL table after update
connection = sqlite3.connect('../data/crypto.db')
cursor = connection.cursor()

cursor.execute ("""
        SELECT *
        FROM bitcoin_prices
        ORDER BY date;
""")

result = cursor.fetchall()

for x in result:
    print(x)

('2025-01-06 21:00:00', 102196.643949755, 102383.400987382, 101640.113923497, 101659.740266516, 11167.1830512779)
('2025-01-06 22:00:00', 101659.740266516, 102112.522349383, 101647.492422707, 102047.45542071, 5065.65861035709)
('2025-01-06 23:00:00', 102047.45542071, 102285.622020661, 101941.107536873, 102280.873054768, 6498.48415850902)
('2025-01-07 00:00:00', 102280.873054768, 102713.731333029, 102006.360074212, 102094.449216145, 12311.7875268933)
('2025-01-07 01:00:00', 102094.449216145, 102119.021951596, 101744.721925884, 101968.617507173, 8433.23999503962)
('2025-01-07 02:00:00', 101968.617507173, 102056.898079246, 101700.468286027, 101726.966625477, 6319.05682950001)
('2025-01-07 03:00:00', 101726.966625477, 101845.514135434, 101574.526776035, 101731.903520634, 6326.98790601675)
('2025-01-07 04:00:00', 101731.903520634, 101951.901873924, 101706.803510684, 101707.120021451, 4531.64285242173)
('2025-01-07 05:00:00', 101707.120021451, 101813.717179016, 101621.196924195, 101759.58844

##### 3. Financial indicators  

a. Queries the data into a DataFrame  
b. Creates a column with dates without time information  
c. Transforms the hourly data into daily data (this helps keeping data granularity in a ordely manner)  
d. Checks the last positions of the DataFrame  
e. Calculates the financial indicators Simple Mean Average (SMA), Average True Range (ATR), Bollinger Bands (BB) and Standard Deviation of the  
returns, saving it back to 'bitcoin_daily_indicators' SQL table.  

In [ ]:
# 3.1 Querying existing data into df

connection = sqlite3.connect('../data/crypto.db') #connects Python to SQLite
cursor = connection.cursor()

existing_data_query = ("""
        SELECT date, open, high, low, close, volume
        FROM bitcoin_prices
        ORDER BY date;
""")

bitcoin_hourly_prices_df = pd.read_sql_query(existing_data_query, connection, parse_dates=['date'])

bitcoin_hourly_prices_df['day'] = bitcoin_hourly_prices_df['date'].dt.date

last_date = bitcoin_hourly_prices_df['day'].max()

bitcoin_hourly_prices_df = bitcoin_hourly_prices_df[bitcoin_hourly_prices_df['day'] < last_date]

bitcoin_daily_prices_df = bitcoin_hourly_prices_df.groupby('day').agg(
    close=('close', 'last'),
    open=('open', 'last'),
    high=('high', 'max'),
    low=('low', 'min'),
    date=('day', 'last'),
    volume=('volume', 'last'),
).reset_index(drop=True)

bitcoin_daily_prices_df = bitcoin_daily_prices_df.set_index('date')

bitcoin_daily_prices_df.tail()


,close,open,high,low,volume
date,,,,,
2025-04-25,95053.349524,95070.540212,95778.630924,92897.633538,4586.891899
2025-04-26,94382.522065,94145.758490,95338.083177,93932.787277,4270.546077
2025-04-27,93681.223198,93015.148977,94566.721416,92881.602296,8883.259669
2025-04-28,94654.804126,94912.556811,95618.866404,93511.744888,7301.618967
2025-04-29,94685.821685,94566.102424,95479.643867,93797.110420,4460.957859


In [ ]:
# 3.2 Calculating financial indicators

windows = [7, 14, 30, 50]  # windows in days

for window in windows:
    bitcoin_daily_prices_df[f'SMA_{window}d'] = calculate_sma(bitcoin_daily_prices_df['close'], window)
    bitcoin_daily_prices_df[f'ATR_{window}d'] = calculate_atr(bitcoin_daily_prices_df[['high', 'low', 'close']], window)
    upper_band, lower_band = calculate_bollinger_bands(bitcoin_daily_prices_df['close'], window)
    bitcoin_daily_prices_df[f'BB_Upper_{window}d'] = upper_band
    bitcoin_daily_prices_df[f'BB_Lower_{window}d'] = lower_band
    bitcoin_daily_prices_df[f'Return_Std_{window}d'] = calculate_returns_std(bitcoin_daily_prices_df['close'], window)

daily_df_to_sql = bitcoin_daily_prices_df.reset_index()

    # 3.2.1 Writing the indicators back to the SQL table, replacing it
connection = sqlite3.connect('../data/crypto.db') #connects Python to SQLite
cursor = connection.cursor()

daily_df_to_sql.to_sql('bitcoin_daily_indicators', connection, if_exists='replace', index=False)

connection.close()

bitcoin_daily_prices_df.tail()

,close,open,high,low,volume,SMA_7d,ATR_7d,BB_Upper_7d,BB_Lower_7d,Return_Std_7d,...,SMA_30d,ATR_30d,BB_Upper_30d,BB_Lower_30d,Return_Std_30d,SMA_50d,ATR_50d,BB_Upper_50d,BB_Lower_50d,Return_Std_50d
date,,,,,,,,,,,,,,,,,,,,,
2025-04-25,95053.349524,95070.540212,95778.630924,92897.633538,4586.891899,90728.858198,2999.520823,98317.959092,83139.757304,1.916905,...,84880.773008,3382.979019,93513.445802,76248.100214,2.831712,84631.180562,3428.985211,91942.138789,77320.222335,2.869902
2025-04-26,94382.522065,94145.758490,95338.083177,93932.787277,4270.546077,92039.531865,3077.849287,98216.579000,85862.484730,2.087336,...,85119.331383,3366.563325,94391.975275,75846.687492,2.838098,84782.367244,3330.090362,92575.115017,76989.619470,2.823668
2025-04-27,93681.223198,93015.148977,94566.721416,92881.602296,8883.259669,92944.197330,2796.285707,97577.044511,88311.350149,2.182029,...,85427.854686,3292.594500,95207.058365,75648.651006,2.767690,84930.114500,3331.684050,93110.321109,76749.907891,2.824517
2025-04-28,94654.804126,94912.556811,95618.866404,93511.744888,7301.618967,93899.801750,2753.982553,95520.938825,92278.664675,2.177558,...,85826.869256,3265.352273,96106.889858,75546.848653,2.730840,85207.301599,3244.447779,93747.008633,76667.594565,2.662232
2025-04-29,94685.821685,94566.102424,95479.643867,93797.110420,4460.957859,94155.784642,2147.609191,95590.940802,92720.628482,0.873225,...,86235.766895,3257.724019,96922.673388,75548.860403,2.727991,85528.489736,3148.946409,94263.719766,76793.259706,2.626484
